# LEGAL-BERT

In [1]:
import os
import optuna
import pandas as pd
import sys
import torch
import transformers
import wandb

sys.path.insert(0, os.path.abspath('../'))

from rhetorical_roles_classification import (
    RhetoricalRolesDataset,
)
from rhetorical_roles_classification.train import train
from rhetorical_roles_classification.test import test_BERT

from config import (
    DATA_FOLDER,
    MODELS_FOLDER,
    HUGGING_FACE_BERT_MODEL,
    NUM_LABELS,
    WANDB_PROJECT,
    MAX_SEGMENT_LENGTH,
    label2rhetRole
)

In [2]:
MODEL_NAME = "LEGAL-BERT"

DEVICE = "cuda:1"

wandb_login_success = wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gmarinowandb-8601. Use `wandb login --relogin` to force relogin


## Dataset Loading

In [3]:
train_dataset = RhetoricalRolesDataset(
    data_filepath=os.path.join(DATA_FOLDER, "train.csv"),
    tokenizer_model_name=HUGGING_FACE_BERT_MODEL,
    max_segment_length=MAX_SEGMENT_LENGTH
)
valid_dataset = RhetoricalRolesDataset(
    data_filepath=os.path.join(DATA_FOLDER, "valid.csv"),
    tokenizer_model_name=HUGGING_FACE_BERT_MODEL,
    max_segment_length=MAX_SEGMENT_LENGTH
)
test_dataset = RhetoricalRolesDataset(
    data_filepath=os.path.join(DATA_FOLDER, "test.csv"),
    tokenizer_model_name=HUGGING_FACE_BERT_MODEL,
    max_segment_length=MAX_SEGMENT_LENGTH
)

## Hyperparameter Tuning

In [4]:
hyperparameters = {
    "batch_size": 128,
    "epochs": 20,
    "num_warmup_steps": 0,
    "accum_iter": 3,
    "early_stopping": True,
    "early_stopping_patience": 2
}

def objective(trial):
    
    print("\n************************ Starting New Optuna Trial ************************\n")
    
    hyperparameters["lr"] = trial.suggest_float("lr", 5e-6, 5e-4)
    hyperparameters["weight_decay"] = trial.suggest_float("weight_decay", 1e-3, 1e-1)

    model = transformers.AutoModelForSequenceClassification.from_pretrained(
        HUGGING_FACE_BERT_MODEL,
        num_labels=NUM_LABELS,
    )
    model_id = f"{MODEL_NAME}-Trial{trial.number}"

    with wandb.init(project=WANDB_PROJECT, config=hyperparameters, name=model_id):

        valid_loss = train(
            model=model,
            destination_path=os.path.join(MODELS_FOLDER, model_id),
            train_dataset=train_dataset,
            valid_dataset=valid_dataset,
            device=DEVICE,
            optuna_trial=trial,
            use_wandb=True,
            **hyperparameters
        )
        
    return valid_loss

In [4]:
study_name = f"{MODEL_NAME}.optuna"
storage_name = "sqlite:///{}.db".format(study_name)
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="minimize")

[I 2023-04-29 12:30:42,093] Using an existing study with name 'LEGAL-BERT.optuna' instead of creating a new one.


In [6]:
study.optimize(objective, n_trials=32)


************************ Starting New Optuna Trial ************************



Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer


************** Training Started **************

EPOCH N. 1


202it [00:42,  4.73it/s]



TRAIN RESULTS
Loss: 0.012451452596860112
Accuracy: 0.5052811432121777
MCC: 0.33069808395025896
Macro F1: 0.22827933217844873
Macro Precision: 0.37020138885059006
Macro Recall: 0.21058942511498954
Micro F1: 0.5052811432121777
Micro Precision: 0.5052811432121777
Micro Recall: 0.5052811432121777

VALIDATION RESULTS
Loss: 0.010323286056518555
Accuracy: 0.6020408163265306
MCC: 0.4857884484761196
Macro F1: 0.4089075013415106
Macro Precision: 0.4247768278176399
Macro Recall: 0.4430565829314329
Micro F1: 0.6020408163265306
Micro Precision: 0.6020408163265306
Micro Recall: 0.6020408163265306
--------------------------------------------------
EPOCH N. 2


...


TRAIN RESULTS
Loss: 0.006921343308644622
Accuracy: 0.7118670394532464
MCC: 0.6285949883687941
Macro F1: 0.5502889669424705
Macro Precision: 0.6178632165196069
Macro Recall: 0.5284038274170841
Micro F1: 0.7118670394532464
Micro Precision: 0.7118670394532464
Micro Recall: 0.7118670394532464

VALIDATION RESULTS
Loss: 0.009515884332358837
Accuracy: 0.6168831168831169
MCC: 0.4965276111612314
Macro F1: 0.44737893793442357
Macro Precision: 0.46964722946420034
Macro Recall: 0.4503918207148366
Micro F1: 0.6168831168831169
Micro Precision: 0.6168831168831169
Micro Recall: 0.6168831168831169
--------------------------------------------------
EPOCH N. 4


202it [00:39,  5.13it/s]



TRAIN RESULTS
Loss: 0.005481286764228222
Accuracy: 0.7725225225225225
MCC: 0.708729058903
Macro F1: 0.6248013094566371
Macro Precision: 0.6839922540717802
Macro Recall: 0.6037134682150466
Micro F1: 0.7725225225225225
Micro Precision: 0.7725225225225225
Micro Recall: 0.7725225225225225

VALIDATION RESULTS
Loss: 0.010216117836534977
Accuracy: 0.6023500309214594
MCC: 0.49071781005649423
Macro F1: 0.4693504447457361
Macro Precision: 0.5157809057052473
Macro Recall: 0.46937449290942984
Micro F1: 0.6023500309214594
Micro Precision: 0.6023500309214594
Micro Recall: 0.6023500309214594
Early stopped training
--------------------------------------------------


Epoch,▁▃▆█
Train Loss,█▄▂▁
Validation Loss,█▁▂▆
Epoch,4
Train Loss,0.00548
Validation Loss,0.01022


[I 2023-04-24 21:01:15,408] Trial 31 finished with value: 0.009431849233806133 and parameters: {'lr': 7.081706652791128e-05, 'weight_decay': 0.0842142337328992}. Best is trial 21 with value: 0.009085902944207191.


In [5]:
print("\n************************ Optuna Study Completed ************************\n")
best_trial = study.best_trial
print("Best trial:", best_trial.number)
print("Value:", best_trial.value)
print("Params:")
for key, value in best_trial.params.items():
    print(f"\t{key}: {value}")


************************ Optuna Study Completed ************************

Best trial: 21
Value: 0.009085902944207191
Params:
	lr: 7.0278184323248e-05
	weight_decay: 0.09155211601590829


## Test

In [6]:
best_trial = study.best_trial
best_params = best_trial.params

model = transformers.AutoModelForSequenceClassification.from_pretrained(
    HUGGING_FACE_BERT_MODEL,
    num_labels=NUM_LABELS,
)
model_id = f"{MODEL_NAME}-Trial{best_trial.number}"
model.load_state_dict(torch.load(os.path.join(MODELS_FOLDER, model_id)))

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

<All keys matched successfully>

In [7]:
test_BERT(
    model=model,
    test_dataset=test_dataset,
    label2rhetRole=label2rhetRole,
    device=DEVICE
)

100%|██████████| 23/23 [00:03<00:00,  7.07it/s]


Overall
	Accuracy: 0.6561306009030914
	MCC: 0.5594308542642985
	Macro F1: 0.4723346093512131
	Macro Precision: 0.5321535407608603
	Macro Recall: 0.45666527835444104
	Micro F1: 0.6561306009030914
	Micro Precision: 0.6561306009030914
	Micro Recall: 0.6561306009030914
Rhetorical role: PREAMBLE
	Macro F1: 0.8368535342545244
	Macro Precision: 0.874517200031333
	Macro Recall: 0.8096196827146922
Rhetorical role: FAC
	Macro F1: 0.7893681195430859
	Macro Precision: 0.7712810714837115
	Macro Recall: 0.8162551934124282
Rhetorical role: RLC
	Macro F1: 0.6334970322033109
	Macro Precision: 0.7031749288174369
	Macro Recall: 0.6019678135959164
Rhetorical role: ISSUE
	Macro F1: 0.8895394343068495
	Macro Precision: 0.9261040334357218
	Macro Recall: 0.8589395546129375
Rhetorical role: ARG_PETITIONER
	Macro F1: 0.6460517504760709
	Macro Precision: 0.6503813463465081
	Macro Recall: 0.6419900320398718
Rhetorical role: ARG_RESPONDENT
	Macro F1: 0.49667832167832165
	Macro Precision: 0.4934004862799583
	Macro 